In [21]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
init_notebook_mode(connected=True)


In [22]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [23]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [24]:
fitness_f

[PosixPath('data/results/transferability_simulation_1/fitness.dat'),
 PosixPath('data/results/transferability_simulation_10/fitness.dat'),
 PosixPath('data/results/transferability_simulation_11/fitness.dat'),
 PosixPath('data/results/transferability_simulation_2/fitness.dat'),
 PosixPath('data/results/transferability_simulation_3/fitness.dat'),
 PosixPath('data/results/transferability_simulation_4/fitness.dat'),
 PosixPath('data/results/transferability_simulation_5/fitness.dat'),
 PosixPath('data/results/transferability_simulation_6/fitness.dat'),
 PosixPath('data/results/transferability_simulation_7/fitness.dat'),
 PosixPath('data/results/transferability_simulation_8/fitness.dat'),
 PosixPath('data/results/transferability_simulation_9/fitness.dat')]

In [25]:
behaviors = read_behaviors([files[2]])[0]

In [ ]:
behaviors.loc[behaviors['genome_id'] == 230].head()

In [26]:
behaviors.describe()

,gen,genome_id,avg_left,avg_right,s1,s2,s3,s4,s5,s6,s7,area0_percentage,area1_percentage,area2_percentage
count,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000
mean,14.109306,292.596750,0.640794,0.730491,0.094463,0.093628,0.085859,0.080622,0.074312,0.068066,0.068839,0.915885,0.038708,0.045406
std,8.884192,177.880133,0.341203,0.359653,0.147592,0.129874,0.115049,0.126255,0.140559,0.170534,0.165759,0.193876,0.147537,0.139072
min,0.000000,1.000000,0.000050,0.000898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140039,0.000000,0.000000
25%,6.000000,133.000000,0.374557,0.446082,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,14.000000,292.000000,0.767948,0.953660,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,22.000000,447.000000,0.936984,0.998736,0.134094,0.150794,0.165842,0.141304,0.106000,0.000000,0.000000,1.000000,0.000000,0.000000
max,29.000000,600.000000,0.999844,1.000000,0.659898,0.795455,0.778518,0.815987,1.000000,1.000000,1.000000,1.000000,0.859961,0.625894


In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
b_grouped.index.get_values()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [27]:
fitness_data = read_moea_fitness([fitness_f[3]])[0]

In [28]:
fitness_data.head()

,gen,genome_id,fitness,str_disparity,diversity
0,0,1,0.061784,0.127241,0.845669
1,0,2,0.061784,0.127241,0.845669
2,0,3,0.061784,0.127241,0.845669
3,0,4,0.061784,0.127241,0.845669
4,0,5,0.061784,0.127241,0.845669


In [29]:
hof = unpickle_moea('data/results/transferability_simulation_11/hof')

In [30]:
len(hof)

16

In [ ]:
plot_moea_fitness(fitness_data, hof)

In [34]:
for ind in hof:
    print(ind.key, ind.fitness.values)

462 (12.631422285116143, 1.5632610793758392, 0.6844539215895322)
587 (12.394281562238861, 0.7622682801127306, 0.6458903093739479)
482 (12.353031310429776, 1.4999352976639826, 0.6619650132709662)
405 (12.341185264100742, 1.171166250826712, 0.7140942988113264)
285 (11.973004488909622, 0.6138094017623146, 0.7177028839556)
456 (11.834505342392706, 1.242976387427235, 0.7448581607816751)
267 (11.805336476454455, 0.9268273000316672, 0.8057048398420568)
586 (11.607311721096101, 0.7622682801127306, 0.7243746644588123)
365 (11.277107265263336, 0.78649120099255, 0.7393310095755793)
201 (11.14380593228393, 0.6086956184004384, 0.5439915722206975)
413 (10.672717479222884, 1.171166250826712, 0.8573173282075575)
148 (10.02513776592615, 0.4970211059195874, 0.7399897863777524)
32 (8.857642204092787, 0.020189515757790266, 0.8724382320982119)
36 (8.224785174691132, 0.020189515757790266, 0.9729083583299559)
34 (4.938600056962514, 0.020189515757790266, 1.013003871013623)
22 (1.0880797338173318, 0.0201895157

In [31]:
genomes_file = sorted(Path('data/results/transferability_simulation_11/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_11/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

In [36]:
transfered_file[:2]

[PosixPath('data/results/transferability_simulation_11/deap_inds/104_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_11/deap_inds/107_transformed_genome_.pkl')]

In [37]:
len(transfered_file)

76

In [38]:
g = sorted(genomes, key=lambda x: x.key)

In [39]:
t = sorted(transferred, key=lambda x: x.key)

In [40]:
sim_genome = list(filter(lambda x: x.key == 405, g))[0]

In [46]:
print(sim_genome.key, sim_genome.fitness.values)

405 (12.341185264100742, 1.171166250826712, 0.7140942988113264)


In [42]:
trans_genome = list(filter(lambda x: x.key == 405, t))[0]

In [47]:
print(trans_genome.key, trans_genome.fitness.values)

405 (12.341185264100742, 1.171166250826712, 0.7140942988113264)


In [ ]:
trans_genome.sim_time

In [44]:
plot_path([sim_genome, trans_genome])

In [ ]:
calc_str_disparity(trans_genome.position, sim_genome.position)

In [48]:
plot_surrogate_model(fitness_data)

In [ ]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    assert s.key == t.key

In [ ]:
str_disparity = Disparity()

In [ ]:
for s, t in union_t:
    print(s.gen, end=" ")

In [ ]:
str_disparities = [(s.gen, s.key, s.fitness.values[1], calc_str_disparity(t.position, s.position)) for s, t in union_t]    

In [ ]:
str_disparities

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [ ]:
plot_str_disparity(filter_disparities)

In [ ]:
np.arange(0, 11)